# Road Following - 道路に沿った走行のデモ(TensorRT版)

Pytorchで学習したモデルをTensorRTモデルに変換したことで高速処理が可能になりました。  
このノートブックでは、TensorRT化したモデルを使うことでカクツキを抑えてJetBotがなめらかに走行することを確認できます。  
Jetson Nano 4GBではこの効果が大きくなりますが、Jetson Nano 2GBではこのTensorRT版でなければまともに走行できないでしょう。

# デバイスの準備

カメラ画像をGPUメモリに転送するために、先に定義だけしておきます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch

########################################
# TensorRTの場合、モデルはGPUを使うように実装されていますが、
# 入力データはGPUメモリに転送する必要があります。
# そのための定義をここでしておきます。
########################################
device = torch.device('cuda')

TensorRTに最適化されたモデル``best_steering_model_xy_trt.pth``を読み込みます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch  # これはすでに読込んでいるため、省略可能です。
from torch2trt import TRTModule  # TensorRTのライブラリを利用します。

########################################
# TensorRTモデルを読み込みます。
########################################
model_trt = TRTModule()  # TensorRTモデルを読み込むための変数を定義します。
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))  # 学習したTensorRTモデルを読み込みます。

## カメラ画像の前処理作成
モデルを読み込みましたが、まだ少し問題があります。  
学習時の入力画像フォーマットと、OpenCVのカメラ画像フォーマットは一致しません。  
これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. HWCをCHWに変換します。
> cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像情報の並び順をHWCからCHW(Channel x Height x Width)に変換します。  
> これはto_tensor()で実行されます。
2. カメラ画像を正規化します。
> これは先に実行されたto_tensor()でRGB値が[0, 255]から[0.0, 1.0]に変換されています。そこにImageNetと同じ正規化パラメータを使ってPytorchのTensorクラスの引き算、割り算機能で正規化しています。
3. カメラ画像をGPUメモリに転送します。
> 入力データはモデルと同じデバイスに存在する必要があります。ここはto_tensor()の行でついでにto(device)という形で実施されています。また、half()でfloat16に変換されているので若干の高速化がされています。
4. 入力画像データを配列に変更します。
> 学習時はバッチサイズ分の画像を配列にして入力層に与えています。モデルの入力層は学習ために入力データを可変長の配列で受け取る構造になっています。そのため予測時は1枚の画像であっても入力画像データを配列にする必要があります。

ここを見たとき、あなたはcollision avoidanceの時と何か違うことに気付くかもしれません。  
えぇ、そうです。カメラ画像はOpenCV経由で取得したものなのでBGRですが、それをRGBに変換していません。  
road followingでは学習時にBGRフォーマットで学習しているため、モデルの入力はBGRのままで最高のパフォーマンスを得ることができます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

########################################
# この値はpytorch ImageNetの学習に使われた正規化（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリングすること）のパラメータです。
# カメラ画像はこの値でRGBを正規化することが望ましいでしょう。
########################################
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

########################################
# カメラ画像をモデル入力用データに変換します。
########################################
def preprocess(image):
    image = PIL.Image.fromarray(image)  # OpenCV画像データ(配列データ）をPILイメージオブジェクトに変換します。
    image = transforms.functional.to_tensor(image).to(device).half()  # 画像をTensor型に変換してfloat16型でGPUメモリに転送します。
    image.sub_(mean[:, None, None]).div_(std[:, None, None])  # ImageNetのパラメータで正規化します。
    return image[None, ...]  # バッチ配列に変換して返します。

> すばらしい、これでカメラ画像をニューラルネットワークの入力フォーマットに変換するための、pre-processing関数を定義できました。　

## カメラの動作確認
それでは、カメラを起動して表示しましょう。  

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

########################################
# カメラを有効化します。
# 画像はwidthとheightで指定したピクセルサイズにリサイズされます。
# fpsのデフォルトは21ですが、カメラフレーム更新に連動して推論を実行するようにコーディングしているため、
# 処理が重くなってしまいます。そのためfpsを小さく設定します。
########################################
camera = Camera(width=224, height=224, fps=4)

In [ ]:
image_widget = ipywidgets.Image()  # 画像表示用のウィジェットを用意します。

########################################
# traitletsライブラリを利用してカメラ画像データが更新されたときに、
# bgr8フォーマットをjpegフォーマットに変換してから
# 画像表示ウィジェットに反映するように設定します。
########################################
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)  # 画像表示ウィジェットをブラウザに表示します。

モーター制御に必要なrobotインスタンスを生成します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import Robot

########################################
# JetBotの制御用クラスをインスタンス化します。
########################################
robot = Robot()

 JetBotの動作パラメータを設定するためのsliderを作成します。
> メモ: テスト時にうまく動作した値でスライダー値を初期化していますが、あなたのデータセットではうまく機能しない可能性があります。そのため、必要に応じてスライダーを増減してセットアップしてください。

## PID制御パラメータ用のスライダーを作成します。
1. ``speed gain``：スピードコントローラー。この値を増やすとJetBotのモーター出力が増加します。
2. ``steering gain``：ステアリングゲインコントローラー。JetBotが左右にブレて不安定な場合は、スムーズに走るようにこの値を減らす必要があります。
3. ``steering kd``：ステアリングディゲインコントローラー。JetBotが左右にブレて不安定な場合は、スムーズに走るようにこの値を増やす必要があります（元の角度を保つパラメータ）
4. ``steering bias``：ステアリングバイアスコントローラー。JetBotがコースの右端または左端に偏って走行する場合は、JetBotが中央のラインまたはコースをたどるまでこのスライダーの値を調整します。ここでは、モーターバイアスと同様にカメラのoffsetも考慮します。

> 注：JetBotの走行をスムーズにするために、上記のスライダーを少しずつ調整します。

* PID制御について
  * https://ja.wikipedia.org/wiki/PID%E5%88%B6%E5%BE%A1

In [ ]:
########################################
# JetBotの動作を調整するためのスライダーウィジェットを用意します。
########################################
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

########################################
# スライダーウィジェットをブラウザに表示します。
########################################
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

次に、JetBotがどう判断しているかを表示するためにいくつかのスライダーを画面に表示しましょう。xおよびyスライダーには、推論されたx, yの値を表示します。

steeringスライダーは推定したステアリング値を表示します。この値はターゲットの実際の角度ではなく、ほぼ比例した単純な値です。  
値が``0``の場合は真っ直ぐ進むことを意味します。

In [ ]:
########################################
# 推論結果とJetBotのステアリング値および速度を表示するためのスライダーを用意します。
########################################
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

########################################
# スライダーウィジェットをブラウザに表示します。
########################################
display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

次は、カメラの画像が変わるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラ画像をPre-processingにかけてモデル入力データに変換する
2. モデル推論の実行
3. ステアリング値を計算する
4. 比例/微分制御（PD）を使用してモーターを制御する

* Tensor型からnumpy.ndarray型に変換する前に、detach()が必要な理由
  * https://stackoverflow.com/questions/63582590/why-do-we-call-detach-before-calling-numpy-on-a-pytorch-tensor

In [ ]:
########################################
# ステアリング計算用の変数を定義します。
########################################
angle = 0.0
angle_last = 0.0

########################################
# カメラ画像が更新されたときに実行する処理を定義します。
########################################
def execute(change):
    # execute()関数はメインスレッドで動作します。このglobal宣言された変数は同じメインスレッドで定義されているため、省略可能です。
    # execute()関数をサブスレッドから呼び出す場合はglobal宣言が必要になります。
    global angle, angle_last
    # カメラ画像を変数xにコピーします。
    image = change['new']
    
    ####################
    # TensorRTモデルで推論を実行して、xとyの値を取得します。
    # カメラ画像をpreprocess(image)で入力用データに変換します。
    # TensorRTモデルで推論を実行します。出力x,yはTensor型に格納されて返ってきます。
    # 数値としてnumpyで扱いたいため、Tensor型からグラフレイヤーを削除するためにdetach()を実行します。
    # （TRTModuleはtorch.nn.Moduleを継承しているため、このdetach()はtorch.nn.Module.detach()になります。）
    # float32型に変換します。
    # 値をCPUメモリに値を転送します。
    # numpy.ndarray()型に変換します。
    # numpy.flatten()でx,yの値を1次配列に変換します。
    ####################
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]  # xの値を取得します。
    y = (0.5 - xy[1]) / 2.0  # yの値を取得します。
    
    ####################
    # x,yの値をスライダーに反映します。
    ####################
    x_slider.value = x
    y_slider.value = y
    
    ####################
    # スピードゲインはそのままスピードとしてスライダーに反映します。
    ####################
    speed_slider.value = speed_gain_slider.value
    
    ####################
    # ステアリング制御のためのPID制御パラメータを求めます。
    ####################
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    ####################
    # ステアリング値をスライダーに反映します。
    ####################
    steering_slider.value = pid + steering_bias_slider.value
    
    ####################
    # 左右モーターの出力を決定します。
    ####################
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})  # execute()関数を1回呼び出して初期化します。

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラ画像の更新に連動して動作させる必要があります。

JetBotでは、traitlets.HasTraitsを継承したCameraクラスを実装しているので、observe()を呼び出すだけで実現できます。

## JetBotを動かしてみよう
次のコードで``start jetbot``ボタンと``stop jetbot``ボタンを作成します。  
``start jetbot``ボタンを押すとモデルの初期化が実行され、JetBotが動作し始めます。  
``speed gain``を0から少しずつ値を大きくし、前進させます。  
``steering gain``でハンドルの切れ角を調整します。0に近いほど、切れ角がゆるくなります。  
最初の1フレームの実行時にメモリの初期化が実行されるので、ディープラーニングではどんなモデルも最初の1フレームの処理はすこし時間がかかります。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import ipywidgets
import time

########################################
# スタートボタンとストップボタンを作成します。
########################################
model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

########################################
# スタートボタンがクリックされた時に呼び出す関数を定義します。
########################################
def start_model(c):
    camera.observe(execute, names='value')  # Cameraクラスのtraitlets.Any()型のvalue変数(カメラ画像データ)が更新されたときに指定した関数を呼び出します。
model_start_button.on_click(start_model)  # startボタンがクリックされた時に指定した関数を呼び出します。

########################################
# ストップボタンがクリックされた時に呼び出す関数を定義します。
########################################    
def stop_model(c):
    camera.unobserve(execute, names='value')  # カメラ画像データの更新と指定した関数の連動を解除します。
    time.sleep(1)  # 実行中の処理が完了するまで少し待ちます。
    robot.stop()  # モーターを停止します。
model_stop_button.on_click(stop_model)  # stopボタンがクリックされた時に指定した関数を呼び出します。

########################################
# ウィジェットの表示レイアウトを定義します。
########################################
model_widget = ipywidgets.VBox([
    ipywidgets.HBox([speed_gain_slider, steering_gain_slider]),
    ipywidgets.HBox([image_widget]),
    steering_slider,
    ipywidgets.HBox([model_start_button, model_stop_button])
])

########################################
# ウィジェットを表示します。
########################################
display(model_widget)

# カメラの停止
データを取り終えたら、他のノートブックでカメラを使用できるようにカメラの接続を適切に閉じましょう。

In [ ]:
camera.stop()  # カメラを停止します。

### 結論
以上がTensorRTのライブデモです。今はJetBotがコース上をなめらかに走行しているのではないでしょうか？　

road followingが上手く行かない場合、正しく走行できるように失敗しやすい場所でさらにデータを追加してください。  
このようにうまくいかない場所を中心にデータを収集すれば、JetBotはさらによく動作するはずです。